In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
class PDE:
    def __init__(self, xi, seed=0):
        """
        xi is a 2d matrix where each entry corresponds to a coefficient of a term, 
        where the 1st axis corresponds to the order of the space partial derivative
        and the 2nd axis corresponds to the degree of the function.
        For example, ut = 1+2u+3u**2 + 4ux+5uxu+6uxu**2 + 7uxx+8uxxu+9uxxu**2
        becomes [[1, 2, 3],[4, 5, 6],[7, 8, 9]]
        """
        
        self.rng = np.random.RandomState(seed)
        
        self.xi = xi
        self.max_order = self.xi.shape[0]-1 #the minus 1 is because it goes from 0 to max, inclusive
        self.max_degree = self.xi.shape[1]-1 #the minus 1 is because it goes from 0 to max, inclusive
        
        self.tpoints = 201
        self.xpoints = 512
        self.trange = (0, 20)
        self.xrange = (-30, 30)
        
        self.dt = (self.trange[1]-self.trange[0])/ self.tpoints #time steps
        self.dx = (self.xrange[1]-self.xrange[0])/ self.xpoints #space step
        
        self.u = self.init_u()
        self.simulated = False
        
    def init_u(self):
        return self.rng.rand(self.xpoints)
    
    def __repr__(self):
        rep = 'ut ='
        for order in range(self.max_order+1):
            for degree in range(self.max_degree+1):
                coef = self.xi[order, degree]
                if coef == 0:
                    term = ''
                else:
                    #coefficient factor
                    if coef > 0:
                        term = ' + ' + str(coef)
                    elif coef < 0:
                        term = ' - ' + str(-coef)
                    #partial deriv factor
                    if order > 0:
                        term += 'u' + 'x'*order
                    else: 
                        pass
                    #function factor
                    if degree == 1:
                        term += 'u'
                    elif degree > 1:
                        term += 'u**'+degree
                    else: 
                        pass
                rep += term
        if rep == 'ut =':
            rep = 'ut = 0'
        return rep
        
        
    def get_dxs(self, point, npoints_behind, npoints_ahead):
        diffs_behind = list(self.dxs[point-npoints_behind:point])
        diffs_ahead = list(self.dxs[point:point+npoints_ahead])
        return diffs_behind + diffs_ahead
    
    def get_du_x(self, time, point):
        ugrid = self.u[time]
        return ugrid[point+1] - ugrid[point]
    def get_dus_x(self, time, point, npoints_behind, npoints_ahead):
        ugrid = self.u[time]
        diffs_behind = []
        for i in range(npoints_behind, 0, -1):
            diffs_behind.append(self.get_du_x(time, point-i))
        diffs_ahead = []
        for i in range(0, npoints_ahead, 1):
            diffs_ahead.append(self.get_du_x(time, point+1))
        return diffs_behind + diffs_ahead
    
    def get_x_derivative(self, time, point, order):
        deriv = self.get_du_x(time, point+order-1) / self.dx**order
        for i in range(order-1, 0, -1):
            deriv -= self.get_x_derivative(time, point+i-1, order-i) / self.dx**i
        return deriv
    def get_t_derivative(self, time, point):
        deriv = 0
        for order in range(self.max_order+1):
            for degree in range(self.max_deg+1):
                coef = self.xi[order, degree]
                xderiv = self.get_x_derivative(time, point, order)
                u = xderiv*self.u[time, point]
                deriv += coef * u**degree * xderiv
        return deriv
    
    def sim_point_step(self, time, point):
        u_curr = self.u[time, point]
        for order in range(self.max_order+1):
            for degree in range(self.max_deg+1):
                u_next = u_curr + dt*self.get_t_derivative(time, point)
    def sim_grid_step(self, time):
        next_grid = np.empty((self.xpoints,), dtype=float)
        for point in range(self.xpoints):
            next_grid[point] = self.sim_point_step(time, point)
        return next_grid
    def sim(self):
        if self.simulated == False:
            spacetime = [self.u]
            for time in range(self.tpoints-1):
                spacetime.append(self.sim_grid_step(time))
            return np.array(spacetime)
        else:
            print("PDE already simulated")
            return self.u

In [14]:
kdv_xi = np.zeros((4, 2))
kdv_xi[3,0] = -1
kdv_xi[1,1] = -6
kdv = PDE(kdv_xi, 0)
print(kdv)
print(kdv.u)
kdv.sim()
print(kdv.u)

ut = - 6.0uxu - 1.0uxxx
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203  0.3595079  0.43703195
 0.6976312  0.06022547 0.66676672 0.67063787 0.21038256 0.1289263
 0.31542835 0.36371077 0.57019677 0.43860151 0.98837384 0.10204481
 0.20887676 0.16130952 0.65310833 0.2532916  0.46631077 0.24442559
 0.15896958 0.11037514 0.65632959 0.13818295 0.19658236 0.36872517
 0.82099323 0.09710128 0.83794491 0.09609841 0.97645947 0.4686512
 0.97676109 0.60484552 0.73926358 0.03918779 0.28280696 0.12019656
 0.2961402  0.11872772 0.31798318 0.41426299 0.0641475  0.69247212
 0.56660145 0.26538949 0.52324805 0.09394

IndexError: too many indices for array

### TO DO
- make sure there aren't issues with boundary conditions when calculating derivatives (maybe make the first xpoints use forward euler step while the last xpoints use backwards euler step)
- rewrite get_x_derivative to have less nested steps
- expand possible equations to include non-linearity, like ut = sin(ux)